## Interference and Inhibition meta analysis
#### Code written by Scott Isherwood, Max Keuken

    Goal of the notebook is to create a number of txt files that can be used as an input for gingerALE. 

In [1]:
# Import standard modules
import shutil, os, re
from glob import glob as glob
import pandas as pd
import numpy as np
from os.path import join as opj

# change width of notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# change max row pandas will spit out
pd.set_option('display.max_rows', 5000)
pd.options.display.max_colwidth = 1000

In [7]:
# Ensure correct wokring directory
if os.getcwd().split('/')[-1].startswith('interference') == True:
    print('In the correct working directory: ' + str(os.getcwd()))
else:
    os.chdir("..")
    print('Changed to correct working directory: ' + str(os.getcwd()))

# Define important working directories
txt_input = opj(os.getcwd(), 'finished_fulltexts/CoordinateTxtFiles') # original txts
txt_output = opj(os.getcwd(), 'Gingerale/GingerALE-Input') # where the contrast files should be

In the correct working directory: /Users/scotti/surfdrive/Projects/interference_in_the_subcortex


In [13]:
# Get all the single coordinates as list
file_list = glob(opj(txt_input, '*.txt'))

#####
# Define the different contrasts by selecting the corresponding txt files
#
# Inhibition contrast
#. The inhibition contrasts contains the SST and the GoNoGo tasks where we look at
#. Succesfull stop > succesfull go & NoGo > Go trials:
contrastInhibition = {s for s in file_list for xs in ['StopS>GoS', 'NoGo>Go'] if xs in s}
print('\nWe have the following number of contrasts included for the inhibition contrasts:', len(contrastInhibition))
#
# Interference contrasts 
#. The interference contrasts contains the Flanker, Simon, Stroop and MISTs tasks
#. where we look at the incongruent > congruent trials:
contrastInterference = {s for s in file_list for xs in ['Inc>Con'] if xs in s}
print('\nWe have the following number of contrasts included for the interference contrasts:', len(contrastInterference))

# Interference and Inhibition contrasts
contrastInhibitionAndInterference = {s for s in file_list for xs in ['StopS>GoS', 'NoGo>Go', 'Inc>Con'] if xs in s}
print('\nWe have the following number of contrasts included for the both inhibition and interference contrasts:', len(contrastInhibitionAndInterference))

# Stop signal studies
contrastSST = {s for s in file_list for xs in ['StopS>GoS'] if xs in s}
print('\nWe have the following number of contrasts included for the SST contrast:', len(contrastSST))

# Go No Go studies
contrastGoNoGo = {s for s in file_list for xs in ['NoGo>Go'] if xs in s}
print('\nWe have the following number of contrasts included for the Go No Go contrast:', len(contrastGoNoGo))

#####
# Concatenate all the txt files that are part of the contrast
#. while starting each txt file on a new line.
contrasts = [contrastInhibition, contrastInterference, contrastInhibitionAndInterference, contrastSST, contrastGoNoGo]
outputfiles = ['contrastInhibitionInput.txt', 'contrastInterferenceInput.txt', 'contrastInhibitionAndInterferenceInput.txt', 'contrastSSTInput.txt', 'contrastGoNoGoInput.txt']

# For the given contrasts concatenate the different studies in a single txt file 
#. what then is used as an input file for gingerALE:
for i in range(0, len(contrasts)):
    #with open(opj('./GingerALE-Input/', outputfiles[i]), 'wb') as outputFile:
    with open(opj(txt_output, outputfiles[i]), 'wb') as outputFile:
        for file in contrasts[i]:
            with open(file,'rb') as fd:
                shutil.copyfileobj(fd, outputFile)
            outputFile.write(b'\n')


We have the following number of contrasts included for the inhibition contrasts: 44

We have the following number of contrasts included for the interference contrasts: 31

We have the following number of contrasts included for the both inhibition and interference contrasts: 75


# GingerALE

In [15]:
# GingerALE via terminal
# Requires the GingerALE.jar file
# Change directory
os.chdir('./Gingerale')

In [16]:
# List inputs required for single dataset ALE functions
!java -cp GingerALE.jar org.brainmap.meta.getALE2

getALE2 usage:
	Foci_Text - name of text file with foci data
optional parameters:
	-mask=Mask_Image
	-ale=ALE_Prefix
	-pval=PVal_Prefix
	-nonadd - use Turkeltaub's nonadditive method
	-noPVal - don't create a P value result image
Thresholding Method, chose at most one:
(If none chosen, no thresholded image will be saved)
	-p=value - Uncorrected P Value
	-pN=value - FDR pN Value
	-pID=value - FDR pID Value
Permutation-Based Thresholding Method:
	-perm=Number of Permutations
	-fwe=value - Family-wise Error Rate
	-clust=value - Cluster-level Inference Rate
Minimum Clust Volume (mm^3):
	(Appropriate with all methods except Cluster-level Inference)
	-minVol=value - Cubic millimeters of the smallest allowed cluster


In [44]:
# FOR INHIBITION AND INTERFERENCE CONTRAST
!java -cp GingerALE.jar org.brainmap.meta.getALE2 \
./GingerALE-Input/contrastInhibitionAndInterferenceInput.txt -mask=MNI_wb.nii \
#-ale=./GingerALE-Output-CLI/contrastInhibitionAndInterferenceOutput_p001_C001_1_ALE.nii \
#-pval=./GingerALE-Output-CLI/contrastInhibitionAndInterferenceOutput_p001_C001_1_PVal.nii \
-nonadd -perm=10000 -clust=0.05 -p=0.001


In [43]:
# MUST DO SECOND STEP TO GET TXT FILES
!java -cp GingerALE.jar org.brainmap.meta.getClustersStats \
./GingerALE-Input/contrastInhibitionAndInterferenceInput.txt \
./GingerALE-Input/contrastInhibitionAndInterferenceInput_p001_C001_1_ALE.nii \
./GingerALE-Input/contrastInhibitionAndInterferenceInput_p001_C001_1_clust.nii \
-mni -out=./GingerALE-Output-CLI/contrastInhibitionAndInterferenceInput_p001_C001_1_clust

In [29]:
# FOR INTERFERENCE ONLY CONTRAST
!java -cp GingerALE.jar org.brainmap.meta.getALE2 \
./GingerALE-Input/contrastInterferenceInput.txt -mask=MNI_wb.nii \
-nonadd -perm=10000 -p=0.001 -fwe=0.05 -minVol=50

FociList.readNextExp(line 7): Subject data found within foci
FociList.readNextExp(line 10): Subject data found within foci
FociList.readNextExp(line 25): Subject data found within foci
FociList.readNextExp(line 38): Subject data found within foci
FociList.readNextExp(line 73): Subject data found within foci
FociList.readNextExp(line 87): Subject data found within foci
FociList.readNextExp(line 95): Subject data found within foci
FociList.readNextExp(line 105): Subject data found within foci
FociList.readNextExp(line 114): Subject data found within foci
FociList.readNextExp(line 122): Subject data found within foci
FociList.readNextExp(line 134): Subject data found within foci
FociList.readNextExp(line 184): Subject data found within foci
FociList.readNextExp(line 230): Subject data found within foci
FociList.readNextExp(line 242): Subject data found within foci
FociList.readNextExp(line 289): Subject data found within foci
FociList.readNextExp(line 321): Subject data found within foci


In [ ]:
# FOR INHIBITION ONLY CONTRAST
!java -cp GingerALE.jar org.brainmap.meta.getALE2 \
./GingerALE-Input/contrastInhibitionInput.txt -mask=MNI_wb.nii \
-nonadd -perm=10000 -p=0.001 -fwe=0.05 -minVol=50

In [45]:
# Print input options for contrast functions
java -cp GingerALE.jar org.brainmap.meta.getALE2Contrast

SyntaxError: invalid syntax (<ipython-input-45-30ee2c953c30>, line 2)

In [24]:
# FOR CONTRASTING INHIBITION AND INTERFERENCE. INH>INT
java -cp GingerALE.jar org.brainmap.meta.getALE2Contrast \
contrastInhibitionOutput_ALE.nii contrastInterferenceOutput_ALE.nii _ALE_pooled.nii \
-mask=MNI_wb.nii -nonadd

SyntaxError: invalid syntax (<ipython-input-24-8438fcbf89e3>, line 2)